<a href="https://colab.research.google.com/github/HrithikGit/Real-Time-Object-Detection/blob/main/CV_Real_Time_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task

 - Use any Object Detector to do real world Objection Detection
. Do detection only for : https://drive.google.com/file/d/1dJF_rcf4jxgjoZc3_wLS4hE4ajBLtvSN/view?usp=drive_link

- READ INPUT ONLY 1 FRAME AT A TIME (30 frames / second)

## **Output frame rate (FPS) should be ~ 30.**.

### **DO NOT RUN ANY SCRIPT ON THE GITHUB** (like demo.py inference.py, etc.)

1. Install requiste libraries
2. Import the Models Class (or write copy paste the models here)
3. Download pretrained weights for the model
4. Load The Pretriained the weights to the model class ```(strict = True)```. All weights should match (if something misses), please let me know immediately.
5. Show a forward Pass (no backpass, or training, just inference)



  
## Some suggested approximations:
 - skip some frames (every 4th frame?)
 - Given Use a tracker to avoid detection for each frame
 - Once object is detected, stop detection just track?
 - Once object is detected, we only need 8 frames, intermittent output?   
 - Skip Detection of stationary object? Waste of compute (bounding box position doest change)
 - Only detect relevant things? : people, moving cars, animals, Thats it....  
 - Skip classification? Only detect and save classification computation?







In [ ]:
!pip install torch torchvision torchaudio
!pip install deep_sort_realtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.5 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision
import cv2
import time
import numpy as np
from google.colab.patches import cv2_imshow
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from concurrent.futures import ThreadPoolExecutor
import torchvision.transforms as T
from collections import deque
from deep_sort_realtime.deepsort_tracker import DeepSort


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive
!pwd
%mkdir test_run
%cd /content/drive/MyDrive/test_run


/content/drive/MyDrive
/content/drive/MyDrive
mkdir: cannot create directory ‘test_run’: File exists
/content/drive/MyDrive/test_run


Add Both video to the above file system (test_run folder in your google drive) : https://drive.google.com/file/d/15lNLJuUqpZxQ5eK8JKFDWnFi9MiKuSpr/view?usp=drive_link

https://drive.google.com/file/d/1dJF_rcf4jxgjoZc3_wLS4hE4ajBLtvSN/view?usp=drive_link

In [ ]:
# #Downloading YOLO v7 model from Github
# !git clone https://github.com/WongKinYiu/yolov7.git
# %cd yolov7
# #Installing requirements required for YOLOv7
# !pip install -r requirements.txt


fatal: destination path 'yolov7' already exists and is not an empty directory.
/content/drive/MyDrive/test_run/yolov7
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 59.8 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.14 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.14 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.1.4.231227 requires numpy>=1.26.0; python_version < "3.13", but you have nu

In [ ]:
# Load the YOLOv5 model from PyTorch Hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()

# Set device to GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the classes we are interested in (humans, cars, animals)
target_classes = ['person', 'car','dog', 'deer']

# Function to filter detections for relevant classes
def filter_detections(detections, target_classes):
    filtered = []
    for *box, conf, cls in detections:
        label = model.names[int(cls)]
        if label in target_classes:
            filtered.append((*box, conf, label))
    return filtered

# Function to calculate the distance between two bounding boxes (for detecting stationary objects)
def calculate_iou(box1, box2):
    # Box format: (x1, y1, x2, y2)
    x1, y1, x2, y2 = box1
    x1_, y1_, x2_, y2_ = box2

    inter_x1 = max(x1, x1_)
    inter_y1 = max(y1, y1_)
    inter_x2 = min(x2, x2_)
    inter_y2 = min(y2, y2_)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)

    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_ - x1_) * (y2_ - y1_)

    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

# Initialize a tracker for each detected object
tracker_types = ['CSRT']
tracker_type = tracker_types[0]  # CSRT is good for accurate object tracking

def create_tracker():
    if tracker_type == 'CSRT':
        return cv2.TrackerCSRT_create()
    # Add other tracker types here if needed

# Open the input video file
video_path = '/content/drive/MyDrive/test_run/IP_Security.mp4'   # Replace with your input video path
cap = cv2.VideoCapture(video_path)

# Get video parameters for writing the output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
output_path = '/content/drive/MyDrive/test_run/output_video.mp4'   # Output video path

# Initialize VideoWriter to save the processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

frame_count = 0
detection_interval = 8  # Run detection every 8 frames
trackers = []  # List to store trackers for objects
last_positions = []  # Store last positions of objects to detect stationary objects

# Process each frame from the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1


    # Convert frame to NumPy array
    img_array = np.array(frame)

    # Skip detection and use tracking if trackers are available
    if frame_count % detection_interval == 0 or len(trackers) == 0:
        # Convert the frame from BGR to RGB (YOLOv5 expects RGB)
        img_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

        # Perform detection
        results = model(img_rgb)

        # Extract detections from model results
        detections = results.xyxy[0].cpu().numpy()  # Extract xyxy, confidence, and class predictions

        # Filter detections for relevant classes (humans, cars, animals)
        filtered_detections = filter_detections(detections, target_classes)

        # Reset trackers and last positions
        trackers = []
        last_positions = []

        # Create a tracker for each detected object
        for *box, conf, label in filtered_detections:
            x1, y1, x2, y2 = map(int, box)
            conf = float(conf)

            # Create a new tracker for this object
            tracker = create_tracker()
            bbox = (x1, y1, x2 - x1, y2 - y1)  # Format: (x, y, width, height)
            tracker.init(frame, bbox)

            trackers.append(tracker)
            last_positions.append((x1, y1, x2, y2))

            # Draw bounding box using NumPy array slicing
            cv2.rectangle(img_array, (x1, y1), (x2, y2), (0, 255, 0), 2)
            # Put label text (class name and confidence)
            cv2.putText(img_array, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    else:
        # Update each tracker
        for i, tracker in enumerate(trackers):
            success, bbox = tracker.update(frame)
            if success:
                x1, y1, w, h = map(int, bbox)
                x2 = x1 + w
                y2 = y1 + h

                # Check if the object is stationary by comparing IoU
                if last_positions[i]:
                    iou = calculate_iou(last_positions[i], (x1, y1, x2, y2))
                    if iou > 0.9:
                        # Object is stationary, skip further tracking
                        continue

                # Update last position of the object
                last_positions[i] = (x1, y1, x2, y2)

                # Draw bounding box using NumPy array slicing
                cv2.rectangle(img_array, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Put label text (Tracking)
                cv2.putText(img_array, 'Tracking', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Calculate FPS
    end_time = time.time()
    fps_text = f'FPS: {1 / (end_time - time.time()):.2f}'

    # Put FPS text on the frame
    cv2.putText(img_array, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Write the frame to the output video
    out.write(img_array)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-5 Python-3.10.12 torch-2.4.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: 